# AutoGPT

使用 LangChain 原语（LLMs、PromptTemplates、VectorStores、Embeddings、Tools）实现 https://github.com/Significant-Gravitas/Auto-GPT。

## 设置工具

我们将设置一个带有搜索工具、写入文件工具和读取文件工具的AutoGPT。

In [2]:
# 导入所需的模块
from langchain.agents import Tool
from langchain_community.tools.file_management.read import ReadFileTool
from langchain_community.tools.file_management.write import WriteFileTool
from langchain_community.utilities import SerpAPIWrapper

# 创建一个SerpAPIWrapper对象
search = SerpAPIWrapper()

# 创建一个工具列表
tools = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    WriteFileTool(),  # 写入文件的工具
    ReadFileTool(),   # 读取文件的工具
]

## 设置内存

这里的内存用于存储代理的中间步骤。

In [3]:
# 导入必要的库
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [4]:
# 定义嵌入模型
embeddings_model = OpenAIEmbeddings()

# 初始化向量存储为空
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

# 注释：
# - `OpenAIEmbeddings()` 是一个嵌入模型的类，用于生成文本的嵌入向量。
# - `faiss` 是一个用于高效相似度搜索的库。
# - `IndexFlatL2()` 是一个用于构建L2距离度量的索引结构。
# - `FAISS()` 是一个用于管理嵌入向量的类，它接受一个嵌入查询函数、一个索引结构、一个文档存储和一个配置字典作为参数。
# - `embeddings_model.embed_query` 是一个函数，用于将查询文本转换为嵌入向量。
# - `InMemoryDocstore({})` 是一个空的文档存储对象，用于存储嵌入向量的文档。
# - `{}` 是一个空的配置字典，用于配置嵌入向量管理类。

## 设置模型和AutoGPT

初始化一切！我们将使用ChatOpenAI模型。

In [5]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain_openai import ChatOpenAI

In [6]:

# 创建AutoGPT实例agent，使用from_llm_and_tools方法
agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",  # 设置AI名称为Tom
    ai_role="Assistant",  # 设置AI角色为Assistant
    tools=tools,  # 使用tools作为工具
    llm=ChatOpenAI(temperature=0),  # 使用ChatOpenAI作为llm，并设置temperature为0
    memory=vectorstore.as_retriever(),  # 使用vectorstore模块中的as_retriever方法作为memory
)

# 将agent的verbose属性设置为True
agent.chain.verbose = True

## 运行示例

在这里，我们将使其编写一个关于旧金山的天气报告。

In [ ]:

# 调用 agent 的 run 方法，并传入参数 ["write a weather report for SF today"]
agent.run(["write a weather report for SF today"])



## 聊天历史记忆

除了保存代理程序即时步骤的记忆外，我们还有一个聊天历史记忆。默认情况下，代理程序将使用'ChatMessageHistory'，但可以进行更改。当您希望使用不同类型的记忆时，例如'FileChatHistoryMemory'，这将非常有用。

In [ ]:
from langchain_community.chat_message_histories import FileChatMessageHistory

agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectorstore.as_retriever(),
    chat_history_memory=FileChatMessageHistory("chat_history.txt"),
)